# seq2seq (Sequence-to-Sequence)

In [ ]:
# 모델 시각화 패키지 install
# !pip install pydot pydotplus graphviz

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24576 sha256=67f9c080770c2f6bd25bb50ae6ac21b3b1e152851c8635498c427e51358be56e
  Stored in directory: c:\users\playdata\appdata\local\pip\cache\wheels\77\54\7c\c8077b6151c819495492300386cf9b151a954259d1a658c63b
Successfully built pydotplus


### Embedding Vector 준비

1. 영어 glove 임베딩 (사전 학습) 사용 /6B tokens, 400K vocab, uncased, 100d
2. 한국어 임베딩 (초기 훈련)

In [ ]:
# !gdown 1UK61mxiWzP1XPvJ53KXa7L7-ETbDYKgc

Downloading...
From (original): https://drive.google.com/uc?id=1UK61mxiWzP1XPvJ53KXa7L7-ETbDYKgc
From (redirected): https://drive.google.com/uc?id=1UK61mxiWzP1XPvJ53KXa7L7-ETbDYKgc&confirm=t&uuid=4b7938f9-4b94-4c86-90a1-fabf70bb64e5
To: c:\encore_skn11\07_nlp\07_seq2seq\glove.6B.100d.txt

  0%|          | 0.00/347M [00:00<?, ?B/s]
  0%|          | 524k/347M [00:00<02:57, 1.96MB/s]
  0%|          | 1.05M/347M [00:00<01:57, 2.94MB/s]
  1%|          | 3.15M/347M [00:00<00:41, 8.20MB/s]
  2%|▏         | 5.77M/347M [00:00<00:27, 12.4MB/s]
  2%|▏         | 7.86M/347M [00:00<00:24, 13.7MB/s]
  3%|▎         | 9.44M/347M [00:00<00:29, 11.6MB/s]
  3%|▎         | 11.5M/347M [00:01<00:24, 13.8MB/s]
  4%|▍         | 13.1M/347M [00:01<00:24, 13.7MB/s]
  4%|▍         | 14.7M/347M [00:01<00:28, 11.5MB/s]
  5%|▍         | 16.3M/347M [00:01<00:30, 10.8MB/s]
  5%|▌         | 17.8M/347M [00:01<00:32, 10.0MB/s]
  6%|▌         | 19.4M/347M [00:01<00:30, 10.6MB/s]
  6%|▋         | 22.0M/347M [00:01<00:23, 13

### 학습 데이터 준비

http://www.manythings.org/anki/

eng-kor 짝으로 이루어진 학습 데이터

1. Encoder 입력 데이터 eng
    - encoder_input_eng 준비    `I love you`
2. Decoder 출력 데이터 kor
    - 학습용 teacher-forcing 모델
        - decoder_input_kor     
        - decoder_output_kor
    - 추론용 모델

In [ ]:
# !gdown 1sfIbATWOTpYiJZbl9NClZ9ZhbSvdR1Wz -O eng_kor.txt

Downloading...
From: https://drive.google.com/uc?id=1sfIbATWOTpYiJZbl9NClZ9ZhbSvdR1Wz
To: c:\encore_skn11\07_nlp\07_seq2seq\eng_kor.txt

  0%|          | 0.00/864k [00:00<?, ?B/s]
 61%|██████    | 524k/864k [00:00<00:00, 878kB/s]
100%|██████████| 864k/864k [00:00<00:00, 1.26MB/s]


In [4]:
eng_inputs = []     # 인코더 입력
kor_inputs = []     # 디코더 입력
kor_targets = []    # 디코더 출력

with open('eng_kor.txt', 'r', encoding='utf-8') as f:
    for line in f:
        eng, kor, _ = line.split('\t')

        kor_input = '<sos> ' + kor
        kor_target = kor + ' <eos>'

        eng_inputs.append(eng)
        kor_inputs.append(kor_input)
        kor_targets.append(kor_target)

len(eng_inputs), len(kor_inputs), len(kor_targets)


(5890, 5890, 5890)

In [ ]:
print(eng_inputs[1000:1005])
print(kor_inputs[1000:1005])    # decoder input
print(kor_targets[1000:1005])   # decoder output

["I'm just sleepy.", "I'm not sulking.", "I'm on the list.", 'Is Tom with you?', 'Is anybody home?']
['<sos> 나 졸려.', '<sos> 나 삐친 거 아니야.', '<sos> 나는 명단에 있다.', '<sos> 톰이랑 같이 있어?', '<sos> 누구 집에 있어요?']
['나 졸려. <eos>', '나 삐친 거 아니야. <eos>', '나는 명단에 있다. <eos>', '톰이랑 같이 있어? <eos>', '누구 집에 있어요? <eos>']


### 토큰화
- 인코더(영어): 영문 토크나이저
- 디코더(한글): 국문 토크나이저

In [7]:
VOCAB_SIZE = 10000

### 영문 토큰화

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

eng_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
eng_tokenizer.fit_on_texts(eng_inputs)
eng_inputs_seq = eng_tokenizer.texts_to_sequences(eng_inputs)   # 토크나이저를 통해 잘라준 시퀀스들

eng_inputs_seq[1000:1005]

[[16, 114, 487],
 [16, 32, 1915],
 [16, 49, 6, 1916],
 [7, 3, 45, 5],
 [7, 367, 89]]

In [ ]:
for seq in eng_inputs_seq[1000:1005]:
    print([eng_tokenizer.index_word[index] for index in seq])   # 인덱스로 저장되어 있는 단어를 다시 단어로 바꿔서 출력 # 대문자를 소문자로 바꿔주는 동작도 함께 수행함

["i'm", 'just', 'sleepy']
["i'm", 'not', 'sulking']
["i'm", 'on', 'the', 'list']
['is', 'tom', 'with', 'you']
['is', 'anybody', 'home']


In [11]:
eng_num_words = min(VOCAB_SIZE, len(eng_tokenizer.word_index))

In [ ]:
eng_max_len = max([len(seq) for seq in eng_inputs_seq])       # 시퀀스를 하나씩 꺼내서 길이가 긴 걸 찾아냄
# 토큰화시킨 것들 중 제일 긴 거 구하기

101

### 국문 토큰화

In [ ]:
# filters='' 필터사용안함 (기본값: filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
kor_tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<oov>', filters='')

kor_tokenizer.fit_on_texts(kor_input + kor_targets) # <sos>, <eos> 포함